In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("UDAF")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/09/09 20:54:05 INFO SparkContext: Running Spark version 3.5.0
24/09/09 20:54:05 INFO SparkContext: OS info Linux, 6.8.0-40-generic, amd64
24/09/09 20:54:05 INFO SparkContext: Java version 11.0.24
24/09/09 20:54:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/09 20:54:06 INFO ResourceUtils: ==============================================================
24/09/09 20:54:06 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/09 20:54:06 INFO ResourceUtils: ==============================================================
24/09/09 20:54:06 INFO SparkContext: Submitted application: UDAF
24/09/09 20:54:06 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name

spark: SparkSession = org.apache.spark.sql.SparkSession@1bb190cc
import spark.implicits._

## Простой пример: Среднее

In [4]:
val data1 = Seq(
                ("Michael", 1, 3000),
                ("Michael", 2, 6000),
                ("Andy", 1, 4500),
                ("Andy", 2, 2500),
                ("Justin", 3, 3500),
                ("Berta", 3, 4000))
            .toDF("name", "depId", "salary")

data1.show()

24/09/09 20:54:09 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/09/09 20:54:09 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/UDF%20UDAF/spark-warehouse'.
24/09/09 20:54:10 INFO CodeGenerator: Code generated in 279.395204 ms
24/09/09 20:54:11 INFO CodeGenerator: Code generated in 10.862698 ms
24/09/09 20:54:11 INFO CodeGenerator: Code generated in 16.298351 ms


+-------+-----+------+
|   name|depId|salary|
+-------+-----+------+
|Michael|    1|  3000|
|Michael|    2|  6000|
|   Andy|    1|  4500|
|   Andy|    2|  2500|
| Justin|    3|  3500|
|  Berta|    3|  4000|
+-------+-----+------+



data1: DataFrame = [name: string, depId: int ... 1 more field]

In [5]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Average(var sum: Long, var count: Long)

class MyAverage extends Aggregator[Long, Average, Double] {
  // Начальное значение. Должно соответствовать свойству: любое b + zero = b
  def zero: Average = Average(0L, 0L)
  // Объединение двух значений в новое значение.
  // Для повышения производительности функция может изменять `buffer` и 
  // возвращать его вместо создания нового объекта.
  def reduce(buffer: Average, data: Long): Average = {
    buffer.sum += data
    buffer.count += 1
    buffer
  }
  // Объединение двух промежуточных значения
  def merge(b1: Average, b2: Average): Average = {
    b1.sum += b2.sum
    b1.count += b2.count
    b1
  }
  // Преобразование выходных данных
  def finish(reduction: Average): Double = reduction.sum.toDouble / reduction.count
  // Кодировщик для типа промежуточного значения
  def bufferEncoder: Encoder[Average] = Encoders.product
  // Кодировщик для типа выходного значения
  def outputEncoder: Encoder[Double] = Encoders.scalaDouble
}

defined class Average
defined class MyAverage

In [6]:
val myAverageUDAF = udaf(new MyAverage)

myAverageUDAF: UserDefinedFunction = UserDefinedAggregator(
  ammonite.$sess.cell5$Helper$MyAverage@4865b929,
  ExpressionEncoder(
    BoundReference(0, LongType, false),
    AssertNotNull(
      UpCast(
        GetColumnByOrdinal(0, LongType),
        LongType,
        List("- root class: \"long\"")
      ),
      List("- root class: \"long\"")
    ),
    Long
  ),
  None,
  true,
  true
)

In [7]:
data1
    .groupBy("depId")
    .agg(myAverageUDAF($"salary").as("average_salary"))
    .show()

24/09/09 20:54:14 INFO CodeGenerator: Code generated in 9.322327 ms
24/09/09 20:54:14 INFO DAGScheduler: Registering RDD 3 (show at cell7.sc:3) as input to shuffle 0
24/09/09 20:54:14 INFO DAGScheduler: Got map stage job 0 (show at cell7.sc:3) with 6 output partitions
24/09/09 20:54:14 INFO DAGScheduler: Final stage: ShuffleMapStage 0 (show at cell7.sc:3)
24/09/09 20:54:14 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:54:14 INFO DAGScheduler: Missing parents: List()
24/09/09 20:54:14 INFO DAGScheduler: Submitting ShuffleMapStage 0 (MapPartitionsRDD[3] at show at cell7.sc:3), which has no missing parents
24/09/09 20:54:15 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 28.8 KiB, free 4.5 GiB)
24/09/09 20:54:15 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 13.0 KiB, free 4.5 GiB)
24/09/09 20:54:15 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on ubuntu:40629 (size: 13.0 KiB, free: 4.5 GiB)
2

+-----+--------------+
|depId|average_salary|
+-----+--------------+
|    1|        3750.0|
|    2|        4250.0|
|    3|        3750.0|
+-----+--------------+



In [8]:
data1
    .groupBy($"name")
    .agg(myAverageUDAF($"salary").as("average_salary"))
    .show

24/09/09 20:54:16 INFO CodeGenerator: Code generated in 10.431046 ms
24/09/09 20:54:16 INFO DAGScheduler: Registering RDD 10 (show at cell8.sc:3) as input to shuffle 1
24/09/09 20:54:16 INFO DAGScheduler: Got map stage job 2 (show at cell8.sc:3) with 6 output partitions
24/09/09 20:54:16 INFO DAGScheduler: Final stage: ShuffleMapStage 3 (show at cell8.sc:3)
24/09/09 20:54:16 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:54:16 INFO DAGScheduler: Missing parents: List()
24/09/09 20:54:16 INFO DAGScheduler: Submitting ShuffleMapStage 3 (MapPartitionsRDD[10] at show at cell8.sc:3), which has no missing parents
24/09/09 20:54:16 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 28.8 KiB, free 4.5 GiB)
24/09/09 20:54:16 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 13.0 KiB, free 4.5 GiB)
24/09/09 20:54:16 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on ubuntu:40629 (size: 13.0 KiB, free: 4.5 GiB

+-------+--------------+
|   name|average_salary|
+-------+--------------+
|Michael|        4500.0|
|   Andy|        3500.0|
| Justin|        3500.0|
|  Berta|        4000.0|
+-------+--------------+



## Продвинутый пример: сумма с ограничением

Каждый клиент получает один балл за каждую заказанную единицу товара, но неболее трёх баллов в одном заказе.

In [9]:
val data = Seq(
    ("Jean-Georges", "Perrin", "NC", 1, 300, 1551903533),
    ("Jean-Georges", "Perrin", "NC", 2, 120, 1551903567),
    ("Jean-Georges", "Perrin", "CA" ,4, 75, 1551903599),
    ("Holden", "Karau", "CA" , 6, 37, 1551904299),
    ("Ginni", "Rometty", "NY", 7, 91, 1551916792),
    ("Holden", "Karau", "CA", 4, 153, 1552876129)
).toDF("firstName", "lastName", "state", "quantity", "revenue", "timestamp")

data.show()

24/09/09 20:54:18 INFO CodeGenerator: Code generated in 22.381653 ms
24/09/09 20:54:18 INFO CodeGenerator: Code generated in 6.1743 ms
24/09/09 20:54:18 INFO CodeGenerator: Code generated in 11.791842 ms


+------------+--------+-----+--------+-------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|
+------------+--------+-----+--------+-------+----------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|
|      Holden|   Karau|   CA|       6|     37|1551904299|
|       Ginni| Rometty|   NY|       7|     91|1551916792|
|      Holden|   Karau|   CA|       4|    153|1552876129|
+------------+--------+-----+--------+-------+----------+



data: DataFrame = [firstName: string, lastName: string ... 4 more fields]

In [10]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Buffer(var value: Int) 

class PointAttribution extends Aggregator[Int, Buffer, Int] {
  val MAX_POINT_PER_ORDER = 3
  // Начальное значение. Должно соответствовать свойству: любое b + zero = b
  def zero: Buffer = Buffer(0)
  // Объединение двух значений в новое значение.
  // Для повышения производительности функция может изменять `buffer` и 
  // возвращать его вместо создания нового объекта.
  def reduce(buffer: Buffer, data: Int): Buffer = {
    val outputValue = if (data < MAX_POINT_PER_ORDER) data else MAX_POINT_PER_ORDER
    buffer.value += outputValue
    buffer
  }
  // Объединение двух промежуточных значения
  def merge(b1: Buffer, b2: Buffer): Buffer = {
    b1.value += b2.value
    b1
  }
  // Преобразование выходных данных
  def finish(reduction: Buffer): Int = reduction.value
  // Кодировщик для типа промежуточного значения
  def bufferEncoder: Encoder[Buffer] = Encoders.product
  // Кодировщик для типа выходного значения
  def outputEncoder: Encoder[Int] = Encoders.scalaInt
}

defined class Buffer
defined class PointAttribution

In [11]:
val pointAttribution = new PointAttribution
val pointAttributionUDAF = udaf(pointAttribution)

pointAttribution: PointAttribution = ammonite.$sess.cell10$Helper$PointAttribution@3ebd6943
pointAttributionUDAF: UserDefinedFunction = UserDefinedAggregator(
  ammonite.$sess.cell10$Helper$PointAttribution@3ebd6943,
  ExpressionEncoder(
    BoundReference(0, IntegerType, false),
    AssertNotNull(
      UpCast(
        GetColumnByOrdinal(0, IntegerType),
        IntegerType,
        List("- root class: \"int\"")
      ),
      List("- root class: \"int\"")
    ),
    Int
  ),
  None,
  true,
  true
)

In [12]:
val t1 = System.nanoTime

data
      .groupBy($"firstName", $"lastName", $"state")
      .agg(sum("quantity"), pointAttributionUDAF($"quantity").as("point"))
      .show()

val duration1 = (System.nanoTime - t1) / 1e9d

24/09/09 20:54:19 INFO CodeGenerator: Code generated in 11.818888 ms
24/09/09 20:54:19 INFO DAGScheduler: Registering RDD 17 (show at cell12.sc:5) as input to shuffle 2
24/09/09 20:54:19 INFO DAGScheduler: Got map stage job 4 (show at cell12.sc:5) with 6 output partitions
24/09/09 20:54:19 INFO DAGScheduler: Final stage: ShuffleMapStage 6 (show at cell12.sc:5)
24/09/09 20:54:19 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:54:19 INFO DAGScheduler: Missing parents: List()
24/09/09 20:54:19 INFO DAGScheduler: Submitting ShuffleMapStage 6 (MapPartitionsRDD[17] at show at cell12.sc:5), which has no missing parents
24/09/09 20:54:19 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 28.1 KiB, free 4.5 GiB)
24/09/09 20:54:19 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 12.8 KiB, free 4.5 GiB)
24/09/09 20:54:19 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on ubuntu:40629 (size: 12.8 KiB, free: 4.5

+------------+--------+-----+-------------+-----+
|   firstName|lastName|state|sum(quantity)|point|
+------------+--------+-----+-------------+-----+
|Jean-Georges|  Perrin|   NC|            3|    3|
|Jean-Georges|  Perrin|   CA|            4|    3|
|      Holden|   Karau|   CA|           10|    6|
|       Ginni| Rometty|   NY|            7|    3|
+------------+--------+-----+-------------+-----+



t1: Long = 104861519797981L
duration1: Double = 0.671533804

In [13]:
println(s"duration1 = $duration1")

24/09/09 20:54:20 INFO BlockManagerInfo: Removed broadcast_3_piece0 on ubuntu:40629 in memory (size: 15.7 KiB, free: 4.5 GiB)
24/09/09 20:54:20 INFO BlockManagerInfo: Removed broadcast_5_piece0 on ubuntu:40629 in memory (size: 15.9 KiB, free: 4.5 GiB)
24/09/09 20:54:20 INFO BlockManagerInfo: Removed broadcast_4_piece0 on ubuntu:40629 in memory (size: 12.8 KiB, free: 4.5 GiB)
24/09/09 20:54:20 INFO BlockManagerInfo: Removed broadcast_2_piece0 on ubuntu:40629 in memory (size: 13.0 KiB, free: 4.5 GiB)


duration1 = 0.671533804


Вариант решения этой задачи без UDAF.

In [14]:
val max = pointAttribution.MAX_POINT_PER_ORDER

val t2 = System.nanoTime
data
        .withColumn("point", when($"quantity".$greater(max), max).otherwise($"quantity"))
        .groupBy($"firstName", $"lastName", $"state")
        .agg(sum("quantity"), sum("point").as("point"))
        .show()
        
val duration2 = (System.nanoTime - t2) / 1e9d

24/09/09 20:54:21 INFO CodeGenerator: Code generated in 53.132465 ms
24/09/09 20:54:21 INFO CodeGenerator: Code generated in 10.394832 ms
24/09/09 20:54:21 INFO DAGScheduler: Registering RDD 23 (show at cell14.sc:7) as input to shuffle 3
24/09/09 20:54:21 INFO DAGScheduler: Got map stage job 6 (show at cell14.sc:7) with 6 output partitions
24/09/09 20:54:21 INFO DAGScheduler: Final stage: ShuffleMapStage 9 (show at cell14.sc:7)
24/09/09 20:54:21 INFO DAGScheduler: Parents of final stage: List()
24/09/09 20:54:21 INFO DAGScheduler: Missing parents: List()
24/09/09 20:54:21 INFO DAGScheduler: Submitting ShuffleMapStage 9 (MapPartitionsRDD[23] at show at cell14.sc:7), which has no missing parents
24/09/09 20:54:21 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 38.8 KiB, free 4.5 GiB)
24/09/09 20:54:21 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 17.0 KiB, free 4.5 GiB)
24/09/09 20:54:21 INFO BlockManagerInfo: Added br

+------------+--------+-----+-------------+-----+
|   firstName|lastName|state|sum(quantity)|point|
+------------+--------+-----+-------------+-----+
|Jean-Georges|  Perrin|   NC|            3|    3|
|Jean-Georges|  Perrin|   CA|            4|    3|
|      Holden|   Karau|   CA|           10|    6|
|       Ginni| Rometty|   NY|            7|    3|
+------------+--------+-----+-------------+-----+



max: Int = 3
t2: Long = 104863062705093L
duration2: Double = 0.867179575

In [15]:
println(s"duration1 = $duration1\nduration2 = $duration2\nduration1 - duration2 = ${duration1 - duration2}")

duration1 = 0.671533804
duration2 = 0.867179575
duration1 - duration2 = -0.19564577100000002
